In [1]:
import os
from data_loader.SingleDataLoader import SingleDataLoader
from data_loader.DoubleDataLoader import DoubleDataLoader
from models.LeNet import LeNet
from models.AlexNet import Alexnet
from trainer.train import train_func
from lib.path import get_training_data_dir
from custom_losses.dice import dice_loss, dice_coefficient
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.losses import BinaryFocalCrossentropy, BinaryCrossentropy

2024-04-11 09:00:26.943212: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 09:00:26.943642: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 09:00:27.324965: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 09:00:28.197375: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 09:00:30.528676: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
data_dir = '../data'
train_list = os.path.join(data_dir, 'train_list')
test_list = os.path.join(data_dir, 'test_list')
val_list = os.path.join(data_dir, 'val_list')

In [7]:
DATA_TYPE = 'gist'
DATA_VOXEL_NUM = 10
CLASSIFYING_RULE = 'WaterClassifyingRuleSurface'
LIGAND_POCKET_DEFINER = 'LigandPocketDefinerOriginal'
LIGAND_VOXEL_NUM = 9

training_data_dir = get_training_data_dir(DATA_TYPE, DATA_VOXEL_NUM, CLASSIFYING_RULE, LIGAND_POCKET_DEFINER, LIGAND_VOXEL_NUM)
print(training_data_dir)

/home/ito/research/data/training_data/gist/data_voxel_num_10/LigandPocketDefinerOriginal/ligand_pocket_voxel_num_9/WaterClassifyingRuleSurface/


In [8]:
data_loader = SingleDataLoader(training_data_dir)

In [4]:
# data_loader = DoubleDataLoader(training_data_dir, training_data_dir)

In [9]:
train_data, train_labels = data_loader.load_data(train_list, DATA_VOXEL_NUM)
test_data, test_labels = data_loader.load_data(test_list, DATA_VOXEL_NUM)
val_data, val_labels = data_loader.load_data(val_list, DATA_VOXEL_NUM)

ValueError: cannot reshape array of size 18522 into shape (1,21,21,21,1)

In [11]:
21*21*21 *2

18522

In [6]:
print('Train data shape: ', train_data.shape)
print('Train labels shape: ', train_labels.shape)
print('Test data shape: ', test_data.shape)
print('Test labels shape: ', test_labels.shape)
print('Val data shape: ', val_data.shape)
print('Val labels shape: ', val_labels.shape)

Train data shape:  (2612, 21, 21, 21, 2)
Train labels shape:  (2612,)
Test data shape:  (1146, 21, 21, 21, 2)
Test labels shape:  (1146,)
Val data shape:  (1108, 21, 21, 21, 2)
Val labels shape:  (1108,)


In [15]:
input_shape = (DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, 1)
epochs = 300
batch_size = 32
n_base = 16
learning_rate = 1e-4
early_stopping = 300
BN = False
dropout = 0.4
model_func = LeNet
losses = [BinaryCrossentropy(), dice_loss]
loss= losses[0]
metrics = ['accuracy', dice_coefficient, Recall(), Precision()]
checkpoint_path = "./checkpoints/LIGAND_VOXEL_NUM_8/GR_VOXEL_NUM_10/LeNet/cp-{epoch:04d}.weights.h5"
model_checkpoint = True

In [16]:
pos = train_labels.sum()
neg = train_labels.shape[0] - pos
total = train_labels.shape[0]

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)

{0: 1.0, 1: 1.0}


In [17]:
clf, clf_hist, clf_eval = train_func(
                                    x_train=train_data,
                                    y_train=train_labels,
                                    x_test=test_data,
                                    y_test=test_labels,
                                    x_val=val_data,
                                    y_val=val_labels,
                                    input_shape=input_shape,
                                    model_func=model_func,
                                    loss=loss,
                                    metrics=metrics,
                                    epochs=epochs,
                                    batch_size=batch_size,
                                    n_base=n_base,
                                    learning_rate=learning_rate,
                                    early_stopping=early_stopping,
                                    checkpoint_path=checkpoint_path,
                                    model_checkpoint=model_checkpoint,
                                    class_weight=class_weight,
                                    BN = BN,
                                    dropout=dropout
                                )

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_4 (Conv3D)               │ (None, 21, 21, 21, 16) │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_4 (MaxPooling3D)  │ (None, 10, 10, 10, 16) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_5 (Conv3D)               │ (None, 10, 10, 10, 32) │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_5 (MaxPooling3D)  │ (None, 5, 5, 5, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │       128,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,369 (556.13 KB)

 Trainable params: 142,369 (556.13 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5289 - dice_coefficient: 0.5120 - loss: 0.6929 - precision_2: 0.5360 - recall_2: 0.4967 - val_accuracy: 0.5578 - val_dice_coefficient: 0.5142 - val_loss: 0.6917 - val_precision_2: 0.5530 - val_recall_2: 0.6029
Epoch 2/300
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5330 - dice_coefficient: 0.5184 - loss: 0.6896 - precision_2: 0.5394 - recall_2: 0.5858 - val_accuracy: 0.5704 - val_dice_coefficient: 0.5147 - val_loss: 0.6888 - val_precision_2: 0.5765 - val_recall_2: 0.5307
Epoch 3/300
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5672 - dice_coefficient: 0.5151 - loss: 0.6836 - precision_2: 0.5571 - recall_2: 0.5923 - val_accuracy: 0.5686 - val_dice_coefficient: 0.5261 - val_loss: 0.6862 - val_precision_2: 0.5529 - val_recall_2: 0.7166
Epoch 4/300
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5650 - dice_coefficient: 0.5198 - loss: 0.6822 - precision_2: 0.5508 - recall_2: 0.6256 - val_accuracy: 0.5722

In [ ]:
prediction = clf.predict(test_data)

524/524 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [11]:
prediction.round().sum()

2417.0

In [5]:
precision = 0.756
recall =0.3592

2*precision*recall/(precision+recall)

0.48700717360114776